In [65]:
import numpy as np
import bokeh.plotting
from bokeh.io import output_notebook, export_png
import matplotlib.pyplot as plt
from scipy import optimize
import timeit
plt.style.use('seaborn-white')
output_notebook()
def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="13px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.xaxis.axis_label_text_font_style = "normal"
    p.yaxis.axis_label_text_font_style = "normal"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

## Toy/Common Sense Example

In [18]:
np.random.seed(2022)
dataxis = np.random.normal(0.0, 1.0, size=(20, 1000))
truebetas = .5*np.ones(shape=(20,))
trueyis = np.matmul(truebetas, dataxis)
noisyyi = trueyis + np.random.normal(0.0, .01, size=(1000,))
estbetas = np.matmul(np.matmul(np.linalg.inv(np.matmul(dataxis, np.transpose(dataxis))), dataxis), noisyyi)
npbetas = np.linalg.lstsq(np.transpose(dataxis), noisyyi)


/var/folders/g5/v23_3kpd15d5q_ctg979nhyw0000gn/T/ipykernel_6947/2398149219.py:7: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  npbetas = np.linalg.lstsq(np.transpose(dataxis), noisyyi)


In [21]:
# common sense check
estbetas - npbetas[0]

array([ 3.33066907e-16, -1.22124533e-15,  1.22124533e-15,  7.77156117e-16,
        0.00000000e+00, -3.33066907e-16, -5.55111512e-16,  1.11022302e-16,
        8.88178420e-16,  2.22044605e-15, -7.77156117e-16,  9.99200722e-16,
       -6.66133815e-16, -9.99200722e-16, -7.77156117e-16,  1.05471187e-15,
       -5.55111512e-16, -6.66133815e-16,  6.66133815e-16,  1.11022302e-16])

In [15]:
truebetas

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [23]:
# estimates, residual, rank, singular values
npbetas

(array([0.49993868, 0.49995244, 0.50002119, 0.49995163, 0.50053818,
        0.50013666, 0.50016279, 0.4997237 , 0.50026423, 0.50011576,
        0.49968709, 0.50023897, 0.5000948 , 0.50025292, 0.50015768,
        0.49994561, 0.50010311, 0.50006312, 0.49958429, 0.50019954]),
 array([0.09804681]),
 20,
 array([35.51263614, 35.28247326, 34.38525506, 33.75336607, 33.60044948,
        33.36970139, 32.65366741, 32.24166035, 32.05257335, 31.63423563,
        31.39288118, 31.2149553 , 30.90468757, 30.24674957, 29.87095436,
        29.47436896, 29.14263201, 28.8678923 , 28.50484304, 27.59114546]))

## Multi-Fidelity Example

In [37]:
def highfid(z, a, b):
    return np.sin(z[0]) + a*np.power(np.sin(z[1]), 2) + b*np.power(z[2], 4)*np.sin(z[0])

def lowfid(z, a, b):
    return np.sin(z[0]) + .6*a*np.power(np.sin(z[1]), 2) + 9*b*np.power(z[2], 2)*np.sin(z[0])

def varest(fz, e):
    return np.sum((fz - e)**2)/(fz.shape[0] - 1)

In [39]:
a = 5
b = .1
lowsample = 9070
highsample = 8
budget = 200
exphs = np.zeros(shape=(budget,))
expls = np.zeros(shape=(budget,))
expmfs = np.zeros(shape=(budget,))
alpha = .9455

for i in range(budget):
    z = np.random.uniform(-np.pi, np.pi, size=(3,lowsample))
    exphs[i] = np.mean(highfid(z[:, :highsample], a, b))
    expls[i] = np.mean(lowfid(z, a, b))
    expmfs[i] = exphs[i] + alpha*(expls[i] - np.mean(lowfid(z[:, :highsample], a, b)))

In [64]:
hhist, hedges = np.histogram(exphs, density=True, bins=50)
lhist, ledges = np.histogram(expls, density=True, bins=50)
mfhist, mfedges = np.histogram(expmfs, density=True, bins=50)
p = bokeh.plotting.figure(height=300, width=800, x_axis_label="Expected Values", \
                          y_axis_label="Frequency")
p.quad(top=hhist, bottom=0, left=hedges[:-1], right=hedges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
p.quad(top=lhist, bottom=0, left=ledges[:-1], right=ledges[1:],
           fill_color="seagreen", line_color="white", alpha=0.5)
p.quad(top=mfhist, bottom=0, left=mfedges[:-1], right=mfedges[1:],
           fill_color="maroon", line_color="white", alpha=0.5)
p.title.align = "center"
legend = bokeh.models.Legend(items=[("Expected Values of High-Fidelity Model", [p.line(line_color="navy")]),
                                    ("Expected Values of Low-Fidelity Model", [p.line(line_color="seagreen")]),
                                    ("Expected Values of Multi-Fidelity Model", [p.line(line_color="maroon")])
                                   ], location = "center")
p.add_layout(legend, "right")
bokeh.io.show(style(p))
p.background_fill_color = None
p.border_fill_color = None
export_png(plot, filename="toymultifidelity.png")

In [31]:
.9465*3.53/3.29

1.015545592705167

In [35]:
np.random.uniform(-np.pi, np.pi, size=(3,9070))[:, :8]

array([[ 2.96034789, -0.74546298, -0.82796973, -0.60979114,  2.4964412 ,
        -1.95607354, -0.29051613, -2.85462637],
       [ 2.34173845, -0.64669706, -0.30627578, -1.58574389,  2.86447767,
         1.58017058,  2.32072506, -0.59952331],
       [-1.58483776,  2.92081704,  2.61667552,  1.92841234,  2.40762552,
         2.82950255,  1.32498647,  1.5794105 ]])